In [1]:
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from textblob import TextBlob
import re
import torch

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Track conversation history
conversation_history = []

# Predefined responses for frequently asked questions
predefined_responses = {
    "hello": "Hi there! How can I assist you today?",
    "how are you": "I'm just a program, but I'm here and ready to help you!",
    "bye": "Goodbye! Have a nice day!",
    "what is your name": "I am a GPT-2 based chatbot. What can I call you?"
}

# Function to generate response
def generate_response(prompt, history):
    # Keep track of conversation history
    context = ' '.join(history) + " " + prompt
    inputs = tokenizer.encode(context, return_tensors='pt')
    outputs = model.generate(inputs, max_length=300, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Update conversation history (trim history if too long)
    history.append(prompt)
    history.append(response)
    if len(history) > 10:
        history = history[-10:]
    return response

# Function to perform sentiment analysis
def analyze_sentiment(response):
    blob = TextBlob(response)
    sentiment = blob.sentiment.polarity
    return sentiment

# Filter response based on sentiment
def generate_response_with_sentiment(prompt, history):
    response = generate_response(prompt, history)
    # Retry generation if the response is too negative
    if analyze_sentiment(response) < -0.5:
        response = generate_response(prompt, history)
    return response

# Function to handle predefined intents
def check_predefined_response(user_input):
    cleaned_input = re.sub(r'[^\w\s]', '', user_input.lower())
    if cleaned_input in predefined_responses:
        return predefined_responses[cleaned_input]
    return None

# Streamlit UI for Chatbot
st.title("Chat with GPT-2")

user_input = st.text_input("You: ", "")
if st.button("Send"):
    if user_input:
        # Check if there's a predefined response
        predefined = check_predefined_response(user_input)
        if predefined:
            response = predefined
        else:
            response = generate_response_with_sentiment(user_input, conversation_history)
        
        # Display the response
        st.write(f"GPT-2: {response}")

        # Display conversation history
        conversation_history.append(f"You: {user_input}")
        conversation_history.append(f"GPT-2: {response}")
        if len(conversation_history) > 10:
            conversation_history = conversation_history[-10:]

    if conversation_history:
        st.write("Conversation History:")
        for i, line in enumerate(conversation_history):
            if i % 2 == 0:
                st.write(f"You: {line}")
            else:
                st.write(f"GPT-2: {line}")


ModuleNotFoundError: No module named 'textblob'